In [2]:
!pip install weaviate-client langchain langchain-community tiktoken pypdf rapidocr-onnxruntime

In [4]:
!pip install -Uqq langchain-weaviate langchain-huggingface

In [14]:
from langchain_weaviate import WeaviateVectorStore
import weaviate
from langchain_huggingface import HuggingFaceEmbeddings

weaviate_url = "vup3o4psmqh9ogassfmrg.c0.asia-southeast1.gcp.weaviate.cloud"
weaviate_api_key = "WWFnSnVvTXRPWnAwNk9lbl9tMi81WUVKWTU3NzdYaUNHOHRoRmwxTm0xS2JvcStjc0d4TC9hdGpqNEI0PV92MjAw"

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=weaviate.auth.AuthApiKey(api_key=weaviate_api_key)
)

embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [6]:
!pip install sentence-transformers

In [7]:
!pip install -U langchain-text-splitters

In [17]:
from langchain_community.document_loaders.pdf import PyPDFLoader

loader = PyPDFLoader(file_path="/content/2312.05934v3.pdf", extract_images=True)

documents = loader.load()

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

docs = text_splitter.split_documents(documents=documents)

In [20]:
from langchain_weaviate import WeaviateVectorStore

for doc in docs:
    if hasattr(doc, 'metadata') and doc.metadata:
        cleaned_metadata = {}
        for key, value in doc.metadata.items():
            clean_key = key.replace('.', '_')
            cleaned_metadata[clean_key] = value
        doc.metadata = cleaned_metadata

db = WeaviateVectorStore.from_documents(docs, embeddings, client=weaviate_client)

In [22]:
print(db.similarity_search(query="What is fine-tuning?", k=4))

[Document(metadata={'subject': 'Proceedings of the International Conference on Machine Learning 2024', 'creator': 'LaTeX with hyperref', 'total_pages': 14.0, 'page_label': '3', 'keywords': '', 'trapped': '/False', 'creationdate': datetime.datetime(2024, 1, 31, 1, 37, 51, tzinfo=datetime.timezone.utc), 'ptex_fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': '/content/2312.05934v3.pdf', 'moddate': datetime.datetime(2024, 1, 31, 1, 37, 51, tzinfo=datetime.timezone.utc), 'producer': 'pdfTeX-1.40.25', 'page': 2.0, 'title': '', 'author': ''}, page_content='a relevant auxiliary knowledge base BQ. Our objective is to\ndiscover a transformation, denoted as F, that, when applied,\nwould enhance the knowledge about Q:\nM′ := F (M, BQ) s.t. LM′,Q > LM,Q. (3)\nIn this work, we aim to compare two choices for F: fine-\ntuning and RAG to see which option performs better in this\nproblem.\n3.2. Fine-Tuning\nFine-tuning is the process of adj

In [24]:
print(db.similarity_search("What is an RAG? Explain.", k=4))

[Document(metadata={'subject': 'Proceedings of the International Conference on Machine Learning 2024', 'creator': 'LaTeX with hyperref', 'total_pages': 14.0, 'page_label': '7', 'keywords': '', 'trapped': '/False', 'creationdate': datetime.datetime(2024, 1, 31, 1, 37, 51, tzinfo=datetime.timezone.utc), 'ptex_fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'moddate': datetime.datetime(2024, 1, 31, 1, 37, 51, tzinfo=datetime.timezone.utc), 'source': '/content/2312.05934v3.pdf', 'title': '', 'page': 6.0, 'producer': 'pdfTeX-1.40.25', 'author': ''}, page_content='calculation. More formally, this means that in Equation (1)\nwe say that M(qn) = cn if:\ncn = arg max\nl\n{M(qn∥a1\nn), . . . ,M(qn∥aL\nn )}, (4)\nwhere M(qn∥al\nn) = log PM(qn∥al\nn).\nMMLU Results For each task and model, we compared\nfour approaches: using just the base model, RAG, FT, and fi-\nnally combining FT and RAG by using the fine-tuned model\nas the generator. Furth

In [29]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

In [30]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [33]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [74]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm_model = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    temperature=1.0,
    max_new_tokens=1000
)

model = ChatHuggingFace(llm=llm_model)

In [75]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
retreiver = db.as_retriever()

rag_chain = (
    {"context": retreiver, "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [65]:
rag_chain.invoke("Explain in breif, what is fine-tuning?")

"Fine-tuning is the process of adjusting a pre-trained model on a specific task or dataset to enhance its performance on that task by fine-tuning on that data, typically through supervised learning. It involves distinguishing between different types such as factual and non-factual questions. This technique can incorporate external knowledge during training or inference time, and can improve the model's performance on that set of questions. Fine-tuning algorithms are classified as either supervised or unsupervised. It compares two methods for knowledge injection, fine-tuning and retrieval augmentation (RAG), presented through consistent terminology in Equations 1 and 2, where knowledge is defined for language models in terms of answerable questions. Given text containing relevant information can improve language model's ability to answer questions about that knowledge. The model's performance on related facts is established by understanding what knowledge means for LLMs, but can disting

In [76]:
rag_chain.invoke("What is a LLM")

' LLM stands for Language Model, a type of artificial intelligence model that is trained on a large corpus of text with the goal of predicting the next word in a sequence given the previous words. Fine-tuning involves fine-tuning on a smaller dataset for a specific task, and knowledge injection refers to training an LLM with new information for a specific task. The ability for a model to answer questions about previously unseen facts is called factual knowledge injection. The propensity for memorization in LLMs during training is known, but interpreting knowledge requires context in its training data, shown by previous work. The selection of a clean dataset made specifically for evaluation aims to test LLM proficiency in each task. The measurement of knowledge in LLMs for specific tasks is called fine-tuning, and binary classification of cleaned examples is called knowledge injection for evaluation. Training and retrieval for specific tasks involves the repetition of information in its